<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate synthetic time series data
def generate_data(seq_length, num_samples):
    x = np.linspace(0, 50, seq_length * num_samples + 1)  # Adjusted length to seq_length * num_samples + 1
    y = np.sin(x) + np.random.normal(0, 0.1, len(x))
    x = y[:-1].reshape(num_samples, seq_length, 1)
    y = y[1:].reshape(num_samples, seq_length, 1)
    return torch.Tensor(x), torch.Tensor(y)

x, y = generate_data(seq_length=50, num_samples=1000)
x, y = x.to(device), y.to(device)

# Define a simple Transformer model for time series forecasting
class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_heads, num_layers):
        super(TransformerTimeSeries, self).__init__()
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=n_heads),
            num_layers=num_layers
        )
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, src):
        src = self.input_projection(src)  # Project input to hidden dimension
        src = src.permute(1, 0, 2)  # (batch_size, seq_length, input_dim) -> (seq_length, batch_size, input_dim)
        output = self.encoder(src)
        output = output.permute(1, 0, 2)  # Back to (batch_size, seq_length, hidden_dim)
        return self.linear(output)

# Create DataLoader
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model
model = TransformerTimeSeries(input_dim=1, hidden_dim=64, output_dim=1, n_heads=4, num_layers=2).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    model.train()
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Evaluation on a test set (example, not implemented here)
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for batch_x, batch_y in dataloader:  # Use your test dataloader here
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        output = model(batch_x)
        loss = criterion(output, batch_y)
        test_loss += loss.item()
    test_loss /= len(dataloader)
    print(f"Test Loss: {test_loss}")